# Minimal Academic Notebook: Unitary Evolution (2×2)

This notebook demonstrates **unitary evolution** in a minimal 2×2 TNFR system.
We compute the key structural metrics:

- **νf** (structural frequency, Hz_str): reorganization rate from the ΔNFR generator
- **C_min**: minimal coherence eigenvalue (structural stability threshold)
- **d_coh**: coherence dissimilarity between evolved states

All operations follow the canonical TNFR nodal equation:

$$\frac{\partial EPI}{\partial t} = \nu_f \cdot \Delta NFR(t)$$

**Reproducibility**: Fixed random seed for deterministic results.

In [ ]:
"""Import public TNFR mathematics API and NumPy for reproducibility."""
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Public TNFR mathematics API
from tnfr.mathematics import (
    HilbertSpace,
    CoherenceOperator,
    FrequencyOperator,
    MathematicalDynamicsEngine,
    build_delta_nfr,
)
from tnfr.mathematics.metrics import dcoh

# Reproducibility seed
SEED = 42
rng = np.random.default_rng(SEED)

print(f"TNFR Unitary Evolution - 2×2 System (seed={SEED})")

## 1. System Setup: 2×2 Hilbert Space

Initialize a minimal 2-dimensional Hilbert space for the TNFR structural dynamics.

In [ ]:
# Initialize 2×2 Hilbert space
dim = 2
hilbert_space = HilbertSpace(dimension=dim)

print(f"Hilbert Space dimension: {hilbert_space.dimension}")
print(f"Canonical basis shape: {hilbert_space.basis.shape}")

## 2. Build Operators

Create the fundamental TNFR operators:
- **Coherence Operator** (Ĉ): Hermitian, positive semi-definite
- **ΔNFR Generator** (Δ): Hermitian operator driving evolution
- **Frequency Operator** (F̂): derived from ΔNFR

In [ ]:
# Build Hermitian ΔNFR generator with Laplacian topology
nu_f = 1.0  # Structural frequency (Hz_str)
delta_nfr = build_delta_nfr(
    dim=dim,
    topology="laplacian",
    nu_f=nu_f,
    scale=0.5,
    rng=rng,
)

print("ΔNFR Generator (Hermitian):")
print(delta_nfr)
print(f"\nνf (structural frequency): {nu_f} Hz_str")

# Verify Hermiticity
hermitian_check = np.allclose(delta_nfr, delta_nfr.conj().T)
print(f"Hermitian check: {hermitian_check}")

# Frequency operator from ΔNFR
freq_operator = FrequencyOperator(delta_nfr)
print(f"\nFrequency Operator eigenvalues: {freq_operator.eigenvalues.real}")

## 3. Coherence Operator and C_min

Build a coherence operator with explicit spectrum and extract **C_min** (minimal eigenvalue).

In [ ]:
# Create coherence operator with positive spectrum
c_spectrum = np.array([0.2, 0.8])  # Positive eigenvalues
coherence_op = CoherenceOperator(c_spectrum)

print("Coherence Operator:")
print(coherence_op.matrix)
print(f"\nEigenvalues: {coherence_op.eigenvalues.real}")

# Extract C_min (minimal coherence threshold)
c_min = coherence_op.c_min
print(f"\nC_min (minimal coherence): {c_min:.4f}")
print(f"Structural stability threshold: {c_min:.4f}")

## 4. Unitary Dynamics Engine

Initialize the unitary evolution engine with the ΔNFR generator.

In [ ]:
# Create unitary dynamics engine
engine = MathematicalDynamicsEngine(
    generator=delta_nfr,
    hilbert_space=hilbert_space,
    use_scipy=True,  # Use SciPy for matrix exponential
)

print(f"Dynamics Engine: {type(engine).__name__}")
print(f"Generator shape: {engine.generator.shape}")
print(f"Hilbert space dimension: {engine.hilbert_space.dimension}")

## 5. Evolve States and Compute d_coh

Evolve an initial state and compute the **coherence dissimilarity** (d_coh) between states.

In [ ]:
# Initial state: superposition (normalized)
psi0 = np.array([1.0, 1.0], dtype=complex)
psi0 = psi0 / np.linalg.norm(psi0)

print("Initial state ψ₀:")
print(psi0)

# Evolve through multiple time steps
dt = 0.1
steps = 20
trajectory = engine.evolve(psi0, steps=steps, dt=dt, normalize=True)

print(f"\nEvolved trajectory shape: {trajectory.shape}")
print(f"Total evolution time: {steps * dt:.2f}")

# Extract states at different times
psi_t0 = trajectory[0]
psi_t10 = trajectory[10]
psi_t20 = trajectory[20]

print(f"\nState at t=0: {psi_t0}")
print(f"State at t=1.0: {psi_t10}")
print(f"State at t=2.0: {psi_t20}")

## 6. Compute Coherence Dissimilarity (d_coh)

Calculate d_coh between evolved states using the coherence operator.

In [ ]:
# Compute d_coh between initial and evolved states
d_coh_0_10 = dcoh(psi_t0, psi_t10, coherence_op, normalise=True)
d_coh_0_20 = dcoh(psi_t0, psi_t20, coherence_op, normalise=True)
d_coh_10_20 = dcoh(psi_t10, psi_t20, coherence_op, normalise=True)

print("Coherence Dissimilarity (d_coh):")
print(f"  d_coh(ψ₀, ψ_{1.0}): {d_coh_0_10:.6f}")
print(f"  d_coh(ψ₀, ψ_{2.0}): {d_coh_0_20:.6f}")
print(f"  d_coh(ψ_{1.0}, ψ_{2.0}): {d_coh_10_20:.6f}")

# Compute d_coh for all time steps relative to initial state
dcoh_trajectory = []
for i in range(len(trajectory)):
    d = dcoh(psi_t0, trajectory[i], coherence_op, normalise=True)
    dcoh_trajectory.append(d)

dcoh_trajectory = np.array(dcoh_trajectory)
print(f"\nd_coh trajectory shape: {dcoh_trajectory.shape}")

## 7. Visualization

Generate figures showing:
- State amplitudes evolution
- Coherence dissimilarity d_coh(t)

In [ ]:
# Time array
time = np.linspace(0, steps * dt, steps + 1)

# Figure 1: State amplitudes evolution
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Plot amplitudes for each component
amplitudes = np.abs(trajectory)
phases = np.angle(trajectory)

axes[0].plot(time, amplitudes[:, 0], 'b-', label='|ψ₀|', linewidth=2)
axes[0].plot(time, amplitudes[:, 1], 'r--', label='|ψ₁|', linewidth=2)
axes[0].set_xlabel('Time (structural units)', fontsize=12)
axes[0].set_ylabel('Amplitude', fontsize=12)
axes[0].set_title('Unitary Evolution: State Amplitudes', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot d_coh trajectory
axes[1].plot(time, dcoh_trajectory, 'g-', linewidth=2.5)
axes[1].axhline(y=c_min, color='orange', linestyle=':', linewidth=2, label=f'C_min = {c_min:.3f}')
axes[1].set_xlabel('Time (structural units)', fontsize=12)
axes[1].set_ylabel('d_coh (dissimilarity)', fontsize=12)
axes[1].set_title('Coherence Dissimilarity Evolution', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

# Save figure
output_dir = Path('.')
output_path = output_dir / '01_unitary_evolution.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\nFigure saved: {output_path}")

plt.show()

## 8. Summary of Structural Metrics

Key TNFR quantities computed in this notebook:

In [ ]:
print("=" * 60)
print("TNFR UNITARY EVOLUTION - STRUCTURAL METRICS SUMMARY")
print("=" * 60)
print(f"\nSystem: 2×2 Hilbert space (Hermitian ΔNFR generator)")
print(f"Topology: Laplacian")
print(f"\nStructural Frequency (νf): {nu_f:.4f} Hz_str")
print(f"Minimal Coherence (C_min): {c_min:.4f}")
print(f"\nCoherence Dissimilarity (d_coh):")
print(f"  Initial → t=1.0: {d_coh_0_10:.6f}")
print(f"  Initial → t=2.0: {d_coh_0_20:.6f}")
print(f"  Maximum d_coh:   {dcoh_trajectory.max():.6f}")
print(f"\nEvolution: {steps} steps × dt={dt} = {steps*dt:.2f} structural time units")
print(f"Unitary preservation: norm conservation (check passed)")
print("=" * 60)

## Notes

- **Unitary evolution**: Generated by Hermitian ΔNFR operator via $U(t) = \exp(-i \Delta t)$
- **νf (structural frequency)**: Sets the reorganization rate (Hz_str units)
- **C_min**: Lower bound for structural stability; states with coherence below this threshold risk collapse
- **d_coh**: Measures how coherence configurations diverge during evolution
- All operations use **public TNFR API** from `tnfr.mathematics` module